# Grouping Data

Grouping data by some attribute is among the most useful and commonly used techniques in data analysis.

Doing so allows us to apply a variety of calculations to each group. 

For example, we could group all the students in 2nd grade by their teacher, and then calculate the median test scores for each group of students to determine if a certain class is falling behind, on track, or excelling.

Grouping data is not peculiar to Python and pandas. It has correllaries in a wide variety of tools such as spreadsheet (Excel Pivot Tables) and databases (SQL GROUP BY statements).

So how do we use it in pandas? 

Let's start with a simple, toy set of data to ensure we wrestle with the key concepts.

In [ ]:
people = [
    {'name': 'John Doe', 'city': 'New York', 'state': 'NY', 'salary': 50000},
    {'name': 'Jane Smith', 'city': 'Los Angeles', 'state': 'CA', 'salary': 65000},
    {'name': 'Michael Johnson', 'city': 'Chicago', 'state': 'IL', 'salary': 40000},
    {'name': 'Emily Davis', 'city': 'Chicago', 'state': 'IL', 'salary': 480000},
    {'name': 'David Wilson', 'city': 'Los Angeles', 'state': 'CA', 'salary': 60000},
    {'name': 'Sarah Brown', 'city': 'Philadelphia', 'state': 'PA', 'salary': 52000},
    {'name': 'Alex Martinez', 'city': 'New York', 'state': 'NY', 'salary': 85000},
    {'name': 'Maria Garcia', 'city': 'New York', 'state': 'NY', 'salary': 160000},
    {'name': 'James Lee', 'city': 'Chicago', 'state': 'IL', 'salary': 80000},
    {'name': 'Linda Harris', 'city': 'San Francisco', 'state': 'CA', 'salary': 100000}
]

First, we'll create a DataFrame from using this data.

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(people)
df

## Grouping - The Hard Way

Let's say we wanted to count the number of people from each state.

If we think through this logically, the very first step would be to group our data by the `state` column. 

You could do this manually or with basic Python as below:

In [ ]:
states = {}
for person in people:
    home_state = person['state']
    try:
        # Access the dictionary by state and add the person to a list
        # This assumes the dictionary already contains a <state> key
        states[home_state].append(person)
    except KeyError:
        # If it's the first time we're encountering the state, 
        # the key will *not* be in the dictionary so we
        # need to add it and set the value a list with our first entry
        states[home_state] = [person]     

Let's import and apply the handy `pprint` function to make it easier to examine the output of our new `states` variable.

In [ ]:
from pprint import pprint
pprint(states)

If you examine this dictionary you can see that each person has been grouped into the appropriate state.

In [ ]:
states['CA']

And you can of course now determine the number of people in each state by counting the length of each state's list.

In [ ]:
for state, people in states.items():
    num = len(people)
    # Use some a one-line condition to determine singular/plural
    person_or_people = 'person' if num == 1 else 'people'
    print(f"{state} has {num} {person_or_people}")

## Grouping with DataFrames

The above strategy of using a dictionary to group people works. But it's many more lines of code than are necessary. Worse, the code can quickly get more convoluted if you wanted to group by multiple columns (e.g. `city` and `state`). 

Instead, the pandas DataFrame provides a handy [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) method that does exactly what its name suggests: group data by the values in one or more columns.

Let's give it a try.

In [ ]:
df.groupby('state')

When we run the above code, we see that we get a `DataFrameGroupBy` object. And that type of object happens to have a [groups](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.groups.html) attribute that let's us examine which rows ended up in each group. 

Let's peek under the hood to see how it did.

In [ ]:
group_obj = df.groupby('state')
group_obj.groups

Hmm...so we're not seeing any actual rows in those groups. 

Can you guess what the numbers in each list signify? 

> HINT: Try printing the original DataFrame stored in the `df` variable.

If you guessed that the integers in each list refer to the index position, or row number, of each record, then congratulations! You are a code sleuth extraordinaire!

In [ ]:
df

## Do stuff with groups

So we now know that when we use `DataFrame.groupby`, *pandas* organizes the data based on the column we specified and simply stores references to each row's position, rather than shuffling around the data itself.

That's all well and good, but generally we want to do *something* with our groups. 

For example, we could count number of people from each state. Notice that we get the same counts as the more lengthy method using Python dictionaries.

> The output is a bit wonky, at least compared to a SQL Group By query, since it includes all the columns in the data.



In [ ]:
group_obj.count()

You can also perform calculations on individual columns, which often makes more sense than applying an aggregate function to all columns in each group.

Here's how we'd count the cities by state using the `name` field.

In [ ]:
group_obj.name.count()

And how we'd find the max and median salary for each state.

> Note we're using `.reset_index()` now to restore the output to a proper DataFrame. We're also using `.rename` to be more explicit about the nature of the calculation.

In [ ]:
max_sal = group_obj.salary.max().reset_index().rename(columns={'salary': 'max_salary'})
max_sal

In [ ]:
med_sal = group_obj.salary.median().reset_index().rename(columns={'salary': 'median_salary'})
med_sal

As an exercise, how would we merge these two DataFrames together (e.g. to do some additional calculations)?

In [ ]:
max_sal.merge(med_sal, on='state')

## Group by multiple columns

You're not limited to just one column when grouping. For example, say you wanted to group by `state` *and* `city`: 

In [ ]:
state_city = df.groupby(by=['state','city']).name.count().reset_index().rename(columns={'name': 'record_count'})
state_city

## Sorting groups

Our new output includes the state, and then as a form of nested data, the cities within those states. 

Finally, let's sort the output from largest to small count.

In [ ]:
state_city.sort_values('record_count', ascending=False)